# Avocado Price Prediction with Neural Network

## Importing modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import tensorflow as tf

## Dataset

### Loading dataset

In [ ]:
df = pd.read_csv('/kaggle/input/avocado-prices-2020/avocado-updated-2020.csv')
df.head()

### Exploratory data analysis

In [ ]:
# looking for missing data
df.isnull().sum()

In [ ]:
# statistics of each column
df.describe().transpose()

In [ ]:
# distribution plot of price
sb.distplot(df['average_price'])

### Features extraction

In [ ]:
# extraction
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].apply(lambda date:date.month)
df = df.drop('date',axis=1)

In [ ]:
def is_organic(type):
    if type == 'organic':
        return 1
    else:
        return 0

In [ ]:
# extraction
df['organic'] = df['type'].apply(is_organic)
df = df.drop('type', axis=1)

In [ ]:
# extraction
df = pd.concat([df, pd.get_dummies(df['geography'])], axis=1)
df = df.drop('geography', axis=1)

In [ ]:
# final dataset
df.head()

## Scaling and splitting dataset

In [ ]:
# inputs and outputs
x = df.drop('average_price',axis=1)
y = df['average_price']

# scaling
scaler = MinMaxScaler()
x = scaler.fit_transform(x)

# splitting
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

## Model

In [ ]:
model = Sequential()

model.add(Dense(120,activation='relu'))
model.add(Dense(60,activation='relu'))
model.add(Dense(30,activation='relu'))
model.add(Dense(15,activation='relu'))
model.add(Dense(7,activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mae')

## Training model

In [ ]:
model.fit(x=x_train, y=y_train, validation_data=(x_test, y_test), batch_size=128, epochs=50)

## History

In [ ]:
history = pd.DataFrame(model.history.history)
history.plot()

## Model evaluation

In [ ]:
predictions = model.predict(x_test)

In [ ]:
plt.plot(y_test, y_test, 'r')
plt.scatter(y_test, predictions)
plt.legend(['perfect predictions', 'model predictions'])

In [ ]:
error = y_test.values - predictions

In [ ]:
sb.distplot(error)

## Save model

In [ ]:
model.save('model.h5')

## Load model

In [ ]:
model = tf.keras.models.load_model('model.h5')